# Analysis

This notebook showcases the data analysis pipeline. It begins by extracting semantic information from communications data to form a structured data set. Following this, various predictive models are applied to the data for analysis. The models are then assessed for their validity and reliability. Lastly, the results are visualised to facilitate interpretation and reporting. 

In [ ]:
#Importing the required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib.ticker import MultipleLocator

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, silhouette_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression

from xgboost import XGBRegressor
from xgboost import XGBClassifier

import time

from statsmodels.iolib.summary2 import summary_col
from statsmodels.stats.diagnostic import normal_ad
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.stattools import durbin_watson

from math import sqrt

from scipy.stats import entropy

from scipy.stats import linregress
import statsmodels.api as sm

import sklearn

In [ ]:
# Starting a timer to identify how long the script takes.
start_time_all = time.time()

In [ ]:
# A function for exporting pandas data tables to markdown.
def pandas_to_markdown(dataframe, filename: str, caption: str, reference: str):
    # Convert the input DataFrame to a Markdown-formatted table, without the index column
    markdown_df = dataframe.to_markdown(index=False)
    
    # Add a metadata string to the Markdown-formatted table containing the caption and reference ID
    metadata_str = "\n" + "\n: " + caption + " {#" + reference + "}"
    markdown_with_metadata = markdown_df + metadata_str
    
    # Write the Markdown-formatted table to a file with the given filename in the "../tables/" directory
    with open("../tables/" + filename + ".md", "w") as text_file:
        text_file.write(markdown_with_metadata)
    
    # Print a confirmation message to the console that the operation is complete
    print("done!")

In [ ]:

# List of libraries included
libraries = [
    'numpy', 'pandas', 'matplotlib', 'seaborn', 'nltk', 'sklearn', 'xgboost', 'statsmodels'
]

# Create a dictionary to store library names and versions
library_versions = {}

# Get version for each library
for library in libraries:
    try:
        library_module = __import__(library)
        version = library_module.__version__
        library_versions[library] = version
    except ImportError:
        library_versions[library] = 'Not installed'

# Create a DataFrame from the dictionary
libraries_versions_df = pd.DataFrame(list(library_versions.items()), columns=['Library', 'Version'])

# Display the DataFrame
libraries_versions_df

#Export to markdown
pandas_to_markdown(libraries_versions_df, "python_libraries_versions", "List of Python libraries and their versions used in the analysis.", "tbl-python-libraries-versions")

In [ ]:
# Define the number of iterations for cross-validation
num_iterations = 10000

In [ ]:
# Import the required data
inc_info = pd.read_csv('../data/final/inc_info.csv')
uprn_data = pd.read_csv('../data/final/uprn/uprn_master.csv')
message_data = pd.read_csv('../data/final/messages.csv')
callsign = pd.read_csv('../data/callsign.csv')
weather = pd.read_csv("../data/final/weather.csv")

In [ ]:
# Merge each of the datasets
inc_df = inc_info.merge(uprn_data, on="uprn", how="inner")
inc_df = inc_df.merge(weather, on="vis_inc_num", how="left")

In [ ]:
# Converting the internal area metric to M2
inc_df["internalArea_m"] = inc_df["internalArea"] * 0.09290303997 # Converting from feet to m2

In [ ]:
# Removing null values from internal area column.
inc_df = inc_df.dropna(subset=["internalArea_m"]) 

In [ ]:
# Caluclating the proportion of fire damage.
inc_df["fire_dam_prop"] = inc_df["fire_damage_area_median_value"] / inc_df["internalArea_m"]

In [ ]:
# Limit values in the columns "fire_dam_prop" and "other_dam_prop" to a maximum of 1
inc_df["fire_dam_prop"] = np.clip(inc_df["fire_dam_prop"], 0, 1)

In [ ]:
inc_nums = list(inc_df["vis_inc_num"])

In [ ]:
# Filtering messages to only include those from incidents with data
message_data = message_data[message_data["incident_number"].isin(inc_nums)]

## Entropy

In [ ]:
# Identifying the message time
message_data["message_time"] = pd.to_datetime(message_data["message_time"].str.split('.').str[0])

In [ ]:
# Adding in the callsign data to identify the type of resource being referneced.
message_data = message_data.merge(callsign, left_on = 'source', right_on='callsign')

In [ ]:
# Initating a list of incidents with multiple arrival times. 
multi_arrivals = []
all_arrivals = pd.DataFrame(columns=['incident_number', 'message_time', 'source', 'response_time', 'Type'])

In [ ]:
# Loop to extract these arrivals. 
for i in inc_nums:
    temp_inc_df = message_data[message_data["incident_number"] == i]
    
    temp_start_time = temp_inc_df["message_time"].min()
    
    temp_arrivals = temp_inc_df[(temp_inc_df["message"].str.contains("to 02 - In Attendance")) & (temp_inc_df["operator_id"] == "SYS")]
    
    temp_arrivals = temp_arrivals.drop_duplicates(subset=["source", "message"], keep="first")
    
    if temp_arrivals["source"].nunique() == temp_arrivals.shape[0]:
        
        temp_arrivals["response_time"] = temp_arrivals["message_time"] - temp_start_time
        
        temp_arrivals = temp_arrivals[["incident_number", "message_time", "source", "response_time", 'Type']]
        
        all_arrivals = pd.concat([all_arrivals, temp_arrivals])

        
        
    else:
        multi_arrivals.append(i)
        

    
    del(temp_inc_df)
    del(temp_start_time)

In [ ]:
# Converting the response time to seconds.
all_arrivals['response_seconds'] = all_arrivals['response_time'].dt.total_seconds()

In [ ]:
# Caluclating how many of each resource were in atttendance.
resources = all_arrivals.groupby(['Type', 'incident_number'])['source'].count().reset_index().pivot(columns = 'Type', index='incident_number').fillna(0).reset_index()

resources.columns = resources.columns.droplevel()
resources = resources.rename(columns={'': 'incident_number'})

In [ ]:
# Adding this information to the master dataset
inc_df = inc_df.merge(resources, left_on="vis_inc_num", right_on="incident_number")

In [ ]:
# Listing the response times for each incident number.
grouped_times = all_arrivals.groupby('incident_number')['response_time'].apply(
    lambda x: x.dt.total_seconds().dropna().astype(int).tolist()
)

In [ ]:
# Calcualting the entropy of response times by incident number.
entropies = []
for times in grouped_times:
    counts = np.bincount(times)
    probabilities = counts / np.sum(counts)
    entropies.append(entropy(probabilities, base=2))

# Add the entropies as a new column in the DataFrame
all_arrivals['entropy'] = pd.Series(entropies)

grouped_times = grouped_times.reset_index()
grouped_times["entropy"] = entropies

In [ ]:
# Merging this information back to the master dataset.
inc_df = inc_df.merge(grouped_times, left_on="vis_inc_num", right_on="incident_number")

## Semantic Extraction

In [ ]:
# Identifying messages relating to agencies
agency_data = message_data[message_data["key_message_flag"] == "A"]
agency_data["count"] = 1

In [ ]:
# Counting how many agency messages by each incident number
agency_count = agency_data.groupby("incident_number")["count"].sum().reset_index().rename(columns={"count":"agency_count"})

In [ ]:
# Merging back to the master data.
inc_df = inc_df.merge(agency_count, left_on="vis_inc_num", right_on="incident_number")

In [ ]:
# Extracting messages that were written by hand and formatting them
hand_written = message_data[message_data["message"].str.contains("Informative")]

hand_written["message"] = hand_written["message"].str.lower()

hand_written["message"] = hand_written["message"].str.replace("_", "")

In [ ]:
# Identifying a stopwords list
stop = stopwords.words('english')

# Removing the stop words
hand_written['message_without_stopwords'] = hand_written["message"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

### TF-ID

In [ ]:
# Initalising the TF-IDF model
v = TfidfVectorizer()
x = v.fit_transform(hand_written["message_without_stopwords"])
z = pd.DataFrame(x.toarray(), columns = v.get_feature_names_out())
z.sum().sort_values(ascending=False)[:10]

In [ ]:
# initialize kmeans with 3 centroids
kmeans = KMeans(n_clusters=3, random_state=42)
# fit the model
kmeans.fit(x)
# store cluster labels in a variable
clusters = kmeans.labels_

In [ ]:
# Plotting an elbow plot for the k means clusters
random_state = 42

# Calculate and store inertia (within-cluster sum of squares) for different values of k
inertia_values = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=random_state)
    kmeans.fit(x)
    inertia = kmeans.inertia_
    
    inertia_values.append(inertia)

# Plot the elbow curve
fig, ax = plt.subplots(figsize=(12, 7))
plt.plot(range(1, 11), inertia_values, marker='o')
plt.title('Elbow Curve for KMeans on TF-IDF Data')
plt.xlabel('Number of Clusters (k)', fontsize=12)
plt.ylabel('Inertia', fontsize=12)
plt.xticks(range(1, 11))

plt.savefig("../figures/tf-idf-kmeans-elbow.pdf", dpi=300, facecolor="white", bbox_inches="tight")

plt.show()

In [ ]:
# Grouping the TF-IDF data on the optimal number of clusters, and testing on other models. 
n_clusters = 3

# Split data into training and test sets
# Perform K-means clustering
kmeans = KMeans(n_clusters=n_clusters, random_state=random_state)
kmeans_clusters = kmeans.fit_predict(x)

# Split data into training and test sets
x_train, x_test, kmeans_clusters_train, kmeans_clusters_test = train_test_split(x, kmeans_clusters, test_size=0.2, random_state=random_state)

# Create a Logistic Regression model
logreg = LogisticRegression(random_state=random_state)
logreg.fit(x_train, kmeans_clusters_train)
logreg_clusters = logreg.predict(x_test)

# Create an XGBoost model
xgb = XGBClassifier(random_state=random_state)
xgb.fit(x_train, kmeans_clusters_train)
xgb_clusters = xgb.predict(x_test)

In [ ]:
# Create a DataFrame to hold cluster assignments
classification_df = pd.DataFrame({
    'K-Means': kmeans_clusters_test,
    'Logistic Regression': logreg_clusters,
    'XGBoost': xgb_clusters
})

# Calculate the number of times assignments were the same for each pair of methods
identical_kmeans_logreg_count = (classification_df['K-Means'] == classification_df['Logistic Regression']).sum()
identical_kmeans_xgb_count = (classification_df['K-Means'] == classification_df['XGBoost']).sum()
identical_logreg_xgb_count = (classification_df['Logistic Regression'] == classification_df['XGBoost']).sum()

# Calculate percentages of identical assignments for each pair
identical_kmeans_logreg_percentage = (identical_kmeans_logreg_count / len(classification_df)) * 100
identical_kmeans_xgb_percentage = (identical_kmeans_xgb_count / len(classification_df)) * 100
identical_logreg_xgb_percentage = (identical_logreg_xgb_count / len(classification_df)) * 100

identical_count = (classification_df['K-Means'] == classification_df['Logistic Regression']) & (classification_df['K-Means'] == classification_df['XGBoost'])
identical_percentage = (identical_count.sum() / len(classification_df)) * 100

# Create a DataFrame to store the results
classification_result_df = pd.DataFrame({
    'Pair Comparison': ['K-Means vs. Logistic Regression', 'K-Means vs. XGBoost', 'Logistic Regression vs. XGBoost', 'All Three Methods'],
    'Percentage of Identical Cluster Assignments': [identical_kmeans_logreg_percentage, identical_kmeans_xgb_percentage, identical_logreg_xgb_percentage, identical_percentage]
})
classification_result_df = classification_result_df.round(3)

In [ ]:
# Split data into training and test sets
hand_written_train, hand_written_test = train_test_split(hand_written, test_size=0.2, random_state=random_state)

# Split 'x' data into training and test sets
x_train, x_test = train_test_split(x, test_size=0.2, random_state=random_state)

# Perform K-means clustering on the training data
kmeans = KMeans(n_clusters=n_clusters, random_state=random_state)
kmeans_clusters_train = kmeans.fit_predict(x_train)

# Predict clusters on the test data
kmeans_clusters_test = kmeans.predict(x_test)

# Create a Logistic Regression model
logreg = LogisticRegression(random_state=random_state)
logreg.fit(x_train, kmeans_clusters_train)

# Predict clusters on the test data using the trained Logistic Regression model
logreg_clusters_test = logreg.predict(x_test)

# Create an XGBoost model
xgb = XGBClassifier(random_state=random_state)
xgb.fit(x_train, kmeans_clusters_train)

# Predict clusters on the test data using the trained XGBoost model
xgb_clusters_test = xgb.predict(x_test)

# Add the cluster assignments to the test set of 'hand_written' DataFrame
hand_written_test['K-Means'] = kmeans_clusters_test
hand_written_test['Logistic Regression'] = logreg_clusters_test
hand_written_test['XGBoost'] = xgb_clusters_test

# map clusters to appropriate labels 
cluster_map = {0: "casualty info", 1: "incident info", 2: "after-action"}

# Apply cluster map to cluster columns
for cluster_col in ['K-Means', 'Logistic Regression', 'XGBoost']:
    hand_written_test[cluster_col] = hand_written_test[cluster_col].map(cluster_map)


In [ ]:
# Export a proportion of the cluster assignments for hand validation
proportion_for_validation = 0.1
hand_validation_df = hand_written_test[["message"]].sample(frac=proportion_for_validation, random_state=random_state)
hand_df = hand_written_test.sample(frac=proportion_for_validation, random_state=random_state)

# Add a new 'hand' column to indicate hand-validation
hand_validation_df['hand'] = ""

# Save the hand validation data to an Excel file
hand_validation_df.to_excel('../data/validate/hand_validation_data.xlsx', index=False)


In [ ]:
# Load the hand-validated data from the Excel file
hand_validated_data = pd.read_excel('../data/validate/hand_validation_data_validated.xlsx')

# Calculate error percentage for each clustering method
def calculate_error_percentage(true_clusters, validated_clusters):
    error_count = (true_clusters != validated_clusters).sum()
    error_percentage = (error_count / len(true_clusters)) * 100
    return error_percentage

# Reset index of hand_df
hand_df_reset = hand_df.reset_index(drop=True)

# Calculate error percentage for each clustering method
kmeans_error = calculate_error_percentage(hand_df_reset['K-Means'], hand_validated_data['hand'])
logreg_error = calculate_error_percentage(hand_df_reset['Logistic Regression'], hand_validated_data['hand'])
xgb_error = calculate_error_percentage(hand_df_reset['XGBoost'], hand_validated_data['hand'])

# Print error percentages
print(f"K-Means Error Percentage: {kmeans_error:.2f}%")
print(f"Logistic Regression Error Percentage: {logreg_error:.2f}%")
print(f"XGBoost Error Percentage: {xgb_error:.2f}%")

In [ ]:
# Calculate error percentages
error_data = {
    "Method": ["K-Means", "Logistic Regression", "XGBoost"],
    "Error Percentage": [kmeans_error, logreg_error, xgb_error]
}

error_df = pd.DataFrame(error_data)
error_df = error_df.round(3)

In [ ]:
# Exporting the table results to markdown for reporting.
pandas_to_markdown(error_df, "tf-idf_clustering_error", "Comparison of error percentages for differenet clustering and classifiaciton methods using Manual Validation. Conducted on the manually transcribed incident log messages.", "tbl-tf-idf-clustering-error")

In [ ]:
# apply clusters
hand_written['cluster'] = clusters


# apply mapping
hand_written['cluster'] = hand_written['cluster'].map(cluster_map)

### PCA

In [ ]:
# initialize PCA with 2 components
pca = PCA(n_components=2, random_state=42)
# pass our X to the pca and store the reduced vectors into pca_vecs
pca_vecs = pca.fit_transform(x.toarray())
# save our two dimensions into x0 and x1
x0 = pca_vecs[:, 0]
x1 = pca_vecs[:, 1]

In [ ]:
hand_written['cluster'] = clusters
hand_written['x0'] = x0
hand_written['x1'] = x1

In [ ]:
def get_top_keywords(n_terms):
    """This function returns the keywords for each centroid of the KMeans"""
    df = pd.DataFrame(x.todense()).groupby(clusters).mean() # groups the TF-IDF vector by cluster
    terms = v.get_feature_names_out() # access tf-idf terms
    for i,r in df.iterrows():
        print('\nCluster {}'.format(i))
        print(','.join([terms[t] for t in np.argsort(r)[-n_terms:]])) # for each row of the dataframe, find the n terms that have the highest tf idf score
            
get_top_keywords(10)

In [ ]:
# map clusters to appropriate labels 
cluster_map = {0: "casualty info", 1: "incident info", 2: "after-action"}
# apply mapping
hand_written['cluster'] = hand_written['cluster'].map(cluster_map)

In [ ]:
# set image size
fig, ax = plt.subplots(figsize=(12, 7))
# set a title
#plt.title("TF-IDF + KMeans", fontdict={"fontsize": 18})
# set axes names
plt.xlabel("PC1", fontdict={"fontsize": 16})
plt.ylabel("PC2", fontdict={"fontsize": 16})
# create scatter plot with seaborn, where hue is the class used to group the data
sns.scatterplot(data=hand_written, x='x0', y='x1', hue='cluster', palette="tab10", edgecolor="white", alpha=0.5)

plt.legend(loc="lower right", title ="Cluster")


ax.tick_params(left=False, bottom=False)
ax.set_xticklabels([])
ax.set_yticklabels([])

plt.title("K-Means Cluster Analysis of TF-IDF: PCA Plot")

plt.savefig("../figures/tf-idf-pca.pdf", dpi=300, facecolor="white", bbox_inches="tight")
plt.show()

In [ ]:
# Counting the number of each message type
y1 = hand_written.groupby(["incident_number", "cluster"])["message_time"].nunique().reset_index().rename(columns={"message_time":"count"})

In [ ]:
# Converting it to a pivot table
y1_pivot = y1.pivot_table(values = "count", index = y1["incident_number"], columns = "cluster").fillna(0)

In [ ]:
# Merging back to the master data
inc_df = inc_df.merge(y1_pivot, left_on="vis_inc_num", right_on="incident_number")

# Distribution of Data

Plots describing the distribution of data

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,5))

plt.grid(which='major', color='black', linewidth=0.5, alpha=0.4)
plt.grid(which='minor', color='black', linewidth=0.25, alpha=0.2)

sns.kdeplot(data = inc_df, x = "fire_dam_prop", shade=True)

plt.xlabel("Proportion of Building Damaged to Fire", fontsize=12)
plt.ylabel("Density", fontsize=12)


median_fire_dam = inc_df["fire_dam_prop"].median()
mean_fire_dam = inc_df["fire_dam_prop"].mean()

plt.axvline(x=median_fire_dam, linewidth=2, color='b', label=f"Median: {median_fire_dam:.2f}", alpha=0.5, linestyle=":")
plt.axvline(x=mean_fire_dam, linewidth=2, color='b', label=f"Mean: {mean_fire_dam:.2f}", alpha=0.5, linestyle="--")


ax.xaxis.set_minor_locator(MultipleLocator(0.05))
ax.yaxis.set_minor_locator(MultipleLocator(0.25))

plt.xlim(0,1)

plt.title("Distributuon of Proportion of Building Damaged to Fire")

plt.legend(title="Key")
plt.savefig("../figures/distributions/fire_dam.pdf", dpi=300, facecolor="white", bbox_inches="tight")

plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,5))

plt.grid(which='major', color='black', linewidth=0.5, alpha=0.4)
plt.grid(which='minor', color='black', linewidth=0.25, alpha=0.2)

sns.kdeplot(data = inc_df, x = "fire_damage_area_median_value", shade=True)

plt.xlabel("Area of Building Damaged to Fire ($m^2$)", fontsize=12)
plt.ylabel("Density", fontsize=12)


median_fire_dam_area = inc_df["fire_damage_area_median_value"].median()
mean_fire_dam_area = inc_df["fire_damage_area_median_value"].mean()

plt.axvline(x=median_fire_dam_area, linewidth=2, color='b', label=f"Median: {median_fire_dam_area:.2f}", alpha=0.5, linestyle=":")
plt.axvline(x=mean_fire_dam_area, linewidth=2, color='b', label=f"Mean: {mean_fire_dam_area:.2f}", alpha=0.5, linestyle="--")


ax.xaxis.set_minor_locator(MultipleLocator(6.25))
ax.yaxis.set_minor_locator(MultipleLocator(0.0025))

plt.xlim(0, 225)

plt.title("Distributuon of Area of Building Damaged to Fire")

plt.legend(title="Key")
plt.savefig("../figures/distributions/fire_dam_area.pdf", dpi=300, facecolor="white", bbox_inches="tight")

plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,5))

plt.grid(which='major', color='black', linewidth=0.5, alpha=0.4)
plt.grid(which='minor', color='black', linewidth=0.25, alpha=0.2)

sns.kdeplot(data = inc_df, x = "entropy", shade=True)

plt.xlabel("Response Time Entropy", fontsize=12)
plt.ylabel("Density", fontsize=12)


median_entropy = inc_df["entropy"].median()
mean_entropy = inc_df["entropy"].mean()

plt.axvline(x=median_entropy, linewidth=2, color='b', label=f"Median: {median_entropy:.2f}", alpha=0.5, linestyle=":")
plt.axvline(x=mean_entropy, linewidth=2, color='b', label=f"Mean: {mean_entropy:.2f}", alpha=0.5, linestyle="--")


ax.xaxis.set_minor_locator(MultipleLocator(0.25))
ax.yaxis.set_minor_locator(MultipleLocator(0.1))

plt.xlim(0,5)

plt.title("Distribution of Response Time Entropy")

plt.legend(title="Key")
plt.savefig("../figures/distributions/entropy.pdf", dpi=300, facecolor="white", bbox_inches="tight")

plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,5))

plt.grid(which='major', color='black', linewidth=0.5, alpha=0.4)
plt.grid(which='minor', color='black', linewidth=0.25, alpha=0.2)

sns.kdeplot(data = inc_df, x = "agency_count", shade=True)

plt.xlabel("Count of Agency Messages", fontsize=12)
plt.ylabel("Density", fontsize=12)


median_agency_count = inc_df["agency_count"].median()
mean_agency_count = inc_df["agency_count"].mean()

plt.axvline(x=median_agency_count, linewidth=2, color='b', label=f"Median: {median_agency_count:.2f}", alpha=0.5, linestyle=":")
plt.axvline(x=mean_agency_count, linewidth=2, color='b', label=f"Mean: {mean_agency_count:.2f}", alpha=0.5, linestyle="--")


ax.xaxis.set_minor_locator(MultipleLocator(1))
ax.yaxis.set_minor_locator(MultipleLocator(0.01))

plt.xlim(0,30)

plt.title("Distribution of Count of Agnecy Messages")

plt.legend(title="Key")
plt.savefig("../figures/distributions/agency_count.pdf", dpi=300, facecolor="white", bbox_inches="tight")

plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,5))

plt.grid(which='major', color='black', linewidth=0.5, alpha=0.4)
plt.grid(which='minor', color='black', linewidth=0.25, alpha=0.2)

sns.kdeplot(data = inc_df, x = "precipitation_sum", shade=True)

plt.xlabel("Total Precipitation 7 Days Prior to Incidents (mm)", fontsize=12)
plt.ylabel("Density", fontsize=12)


median_precip = inc_df["precipitation_sum"].median()
mean_precip = inc_df["precipitation_sum"].mean()

plt.axvline(x=median_precip, linewidth=2, color='b', label=f"Median: {median_precip:.2f} mm", alpha=0.5, linestyle=":")
plt.axvline(x=mean_precip, linewidth=2, color='b', label=f"Mean: {mean_precip:.2f} mm", alpha=0.5, linestyle="--")


ax.xaxis.set_minor_locator(MultipleLocator(5))
ax.yaxis.set_minor_locator(MultipleLocator(0.0025))

plt.xlim(0,120)

plt.title("Distribution of Total Precipitation")

plt.legend(title="Key")
plt.savefig("../figures/distributions/precipitation.pdf", dpi=300, facecolor="white", bbox_inches="tight")

plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,5))

plt.grid(which='major', color='black', linewidth=0.5, alpha=0.4)
plt.grid(which='minor', color='black', linewidth=0.25, alpha=0.2)

sns.kdeplot(data = inc_df, x = "mean_temperature", shade=True)

plt.xlabel("Mean Temperature 7 Days Prior to Incidents (°C)", fontsize=12)
plt.ylabel("Density", fontsize=12)


median_temp = inc_df["mean_temperature"].median()
mean_temp = inc_df["mean_temperature"].mean()

plt.axvline(x=median_temp, linewidth=2, color='b', label=f"Median: {median_temp:.2f} °C", alpha=0.5, linestyle=":")
plt.axvline(x=mean_temp, linewidth=2, color='b', label=f"Mean: {mean_temp:.2f} °C", alpha=0.5, linestyle="--")


ax.xaxis.set_minor_locator(MultipleLocator(0.25))
ax.yaxis.set_minor_locator(MultipleLocator(0.025))

plt.title("Distribution of Mean Temperature")

plt.legend(title="Key")
plt.savefig("../figures/distributions/temperature.pdf", dpi=300, facecolor="white", bbox_inches="tight")

plt.show()

## Linear Regression

In [ ]:
# Initialize a timer
start_time = time.time()


# Create lists to store results
coefficients = []
r2_scores = []
r2_score_x = []
model_r2_scores = []
p_values = []
percentage_within_bounds_list = []
real_predicted_values = []
residuals_values = []
rmse_values = []





# Assuming your dataset is in 'inc_df', create the feature matrix (X) and target variable (y)
X = inc_df[['entropy', 'agency_count', 'after-action', 'casualty info', 'incident info', 'mean_temperature', 'precipitation_sum', 'windspeed', 'energyScore']]

# Handle missing values
X.dropna(inplace=True)
y = inc_df['fire_dam_prop'][X.index]

# Perform one-hot encoding for the categorical column (replace 'energyScore' with the actual column name)
X = pd.get_dummies(X, columns=['energyScore'], drop_first=True, dtype=int)

# Add a constant term to the independent variables to account for the intercept in the regression
X = sm.add_constant(X)

for _ in range(num_iterations):
    # Split the data into training and testing sets (80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=np.random.randint(1000))
    
    # Fit the multiple linear regression model
    model = sm.OLS(y_train, X_train).fit()
    
    # Get the summary of the regression model
    summary = model.summary()
    
    # Extract coefficients and p-values
    model_params = model.params
    pvals = model.pvalues
    
    # Get R-squared
    model_r2 = model.rsquared
    
    # Generate predictions for the test set
    test_mean_predictions = model.predict(X_test)
    
    # Calculate R-squared based on predictive ability
    r2_x = 1 - (np.sum((y_test - test_mean_predictions) ** 2) / np.sum((y_test - np.mean(y_test)) ** 2))
    r2 = r2_score(y_test, test_mean_predictions)
    
    # Calculate the residuals for the test set
    residuals = y_test - test_mean_predictions
    
    # Calculate the standard deviation of residuals (a measure of uncertainty)
    residual_std = residuals.std()
    
    # Calculate RMSE for the test set
    rmse = sqrt(mean_squared_error(y_test, test_mean_predictions))

    # Append the RMSE value to the list
    rmse_values.append(rmse)
    
    # Calculate the range of predictions (within one standard deviation) for the test set
    test_lower_bound = test_mean_predictions - residual_std
    test_upper_bound = test_mean_predictions + residual_std
    
    # Count the number of predictions within the bounds
    num_within_bounds = ((y_test >= test_lower_bound) & (y_test <= test_upper_bound)).sum()
    
    # Calculate the percentage of predictions within the bounds
    percentage_within_bounds = num_within_bounds / len(y_test) * 100
    
    # Append results to lists
    coefficients.append(model_params)
    r2_scores.append(r2)
    r2_score_x.append(r2_x)
    model_r2_scores.append(model_r2)
    p_values.append(pvals)
    percentage_within_bounds_list.append(percentage_within_bounds)
    
    # Append real and predicted values to the list
    real_predicted_values.append(pd.DataFrame({'Real': y_test, 'Predicted': test_mean_predictions}))
    
    # Append the residuals to the list
    residuals_values.append(pd.DataFrame({'Residuals': residuals}))

# Create a DataFrame to store real and predicted values from each iteration
real_predicted_df = pd.concat(real_predicted_values, axis=0)

residuals_values_df = pd.concat(residuals_values, axis=0)


# Create DataFrames to store the results
coefficients_df = pd.DataFrame(coefficients)


results_summary = pd.DataFrame({
    'Predictive R-squared': r2_scores,
    'Other Predictive R-squared': r2_score_x,
    'Model R-squared': model_r2_scores,
    'RMSE': rmse_values,  # Here you include the RMSE values
    'Percentage within Bounds': percentage_within_bounds_list
})

# Print or analyze the DataFrames as needed
#coefficients_df
#p_values_df
#results_summary

# Calculate the duration
end_time = time.time()
duration = end_time - start_time

# Print the duration
print("Total duration: {:.2f} seconds".format(duration))

In [ ]:
# Creating a dataframe to compare real and predicted values
real_predicted_df = real_predicted_df[(real_predicted_df["Predicted"] >= 0) & (real_predicted_df["Predicted"] <= 1)]

In [ ]:
# Calculating the proportion of data that is overestimated.
print(str(round(real_predicted_df[real_predicted_df['Predicted'] > real_predicted_df['Real']].shape[0] 
                / real_predicted_df.shape[0] * 100, 2)) +"% overestimated.")

In [ ]:
# Plotting the difference between the predicted and actual data.
fig, ax = plt.subplots(1, figsize=(10,5))

# Calculate the slope and intercept for the line of best fit
slope, intercept = np.polyfit(real_predicted_df['Real'], real_predicted_df['Predicted'], 1)


plt.grid(which='major', color='black', linewidth=0.5, alpha=0.4)
plt.grid(which='minor', color='black', linewidth=0.25, alpha=0.2)

ax.xaxis.set_minor_locator(MultipleLocator(0.05))
ax.yaxis.set_minor_locator(MultipleLocator(0.05))

sns.scatterplot(data = real_predicted_df, x="Real", y="Predicted", alpha=0.05, edgecolor="lightblue", color="None", ax=ax)
plt.plot([0, 1], [intercept, slope + intercept], color='blue', linestyle='-', label= f'Predicted = {slope:.2f} * Real + {intercept:.2f}')

plt.xlabel("Real", fontsize=12)
plt.ylabel("Predicted", fontsize=12)

plt.xlim(0,1)
plt.ylim(0,1)

plt.legend(title="Key")

plt.title('Scatter Plot with Lines of Best Fit for Multilinear regression', y=1.05)
plt.suptitle("Predicting Proportion of Property Damage", y=0.92, size=10)

plt.savefig("../figures/scatter/linear_real_predicted_prop_damage.png", dpi=300, facecolor="white", bbox_inches="tight")

plt.show()

In [ ]:
# Perform the Anderson-Darling test for normal distribution of residuals
p_value = normal_ad(residuals_values_df["Residuals"])[1]
print("p-value from the test - below 0.05 generally means non-normal:", p_value)

# Reporting the normality of the residuals
if p_value < 0.05:
    print("Residuals are not normally distributed")
else:
    print("Residuals are normally distributed")

# Plotting the residuals distribution
fig, ax = plt.subplots(1, figsize=(10,5))
plt.title("Distribution of Residuals")
sns.kdeplot(residuals_values_df["Residuals"], color="lightblue", fill=True, ax=ax)


plt.grid(which='major', color='black', linewidth=0.5, alpha=0.4)
plt.grid(which='minor', color='black', linewidth=0.25, alpha=0.2)

plt.axvline(x=residuals_values_df["Residuals"].median(), linewidth=2, color='b', label=f"Median: {residuals_values_df['Residuals'].median():.2f}", alpha=0.5, linestyle=":")
plt.axvline(x=residuals_values_df["Residuals"].mean(), linewidth=2, color='b', label=f"Mean: {residuals_values_df['Residuals'].mean():.2f}", alpha=0.5, linestyle="--")


ax.xaxis.set_minor_locator(MultipleLocator(0.0625))
ax.yaxis.set_minor_locator(MultipleLocator(0.125))

plt.title("Distribution of risiduals for linear model predicting proportion of building burned")

plt.legend(title="Key")

plt.savefig("../figures/distributions/linear_risiduals_prop_damage.pdf", dpi=300, facecolor="white", bbox_inches="tight")


plt.show()

p_value_thresh=0.05

print()
if p_value > p_value_thresh:
    print('Assumption satisfied')
else:
    print('Assumption not satisfied')
    print()
    print('Confidence intervals will likely be affected')
    print('Try performing nonlinear transformations on variables')


In [ ]:
fig, ax = plt.subplots(1, figsize=(10,5))
sns.heatmap(X.iloc[:,1:].corr(), ax=ax, annot=True, fmt=".2f")
plt.title("Correlation of Variables")

plt.savefig("../figures/linear_correlation_prop_damage.pdf", dpi=300, facecolor="white", bbox_inches="tight")


In [ ]:
# Gathering the VIF for each variable
VIF = [variance_inflation_factor(X.iloc[:,1:], i) for i in range(X.iloc[:,1:].shape[1])]

# Create a DataFrame to store VIF values with variable names
vif_df = pd.DataFrame({
    'Variable': X.iloc[:,1:].columns,
    'VIF': VIF
})

# Round the VIF values to 2 decimal places
vif_df['VIF'] = vif_df['VIF'].round(2)

vif_df

pandas_to_markdown(vif_df, "linear_vif_prop_damage", "Variance Inflation Factors (VIF) table showing multicollinearity among predictor variables. Higher VIF values indicate stronger multicollinearity, potentially affecting the interpretability and reliability of regression coefficients. Note: The values presented are rounded to two decimal places.", "tbl-linear_vif_prop_damage")

In [ ]:
durbinWatson = durbin_watson(residuals_values_df["Residuals"])

# Interpretation based on the Durbin-Watson score
if durbinWatson < 1.5:
    interpretation = 'Signs of positive autocorrelation. Assumption not satisfied. Consider adding lag variables.'
elif durbinWatson > 2.5:
    interpretation = 'Signs of negative autocorrelation. Assumption not satisfied. Consider adding lag variables.'
else:
    interpretation = 'Little to no autocorrelation. Assumption satisfied.'

# Create a DataFrame
autocorrelation_table = pd.DataFrame({
    'Durbin Watson Score': [durbinWatson],
    'Interpretation': [interpretation]
})

# Round the VIF values to 2 decimal places
autocorrelation_table['Durbin Watson Score'] = autocorrelation_table['Durbin Watson Score'].round(2)

# Print the DataFrame
autocorrelation_table

pandas_to_markdown(autocorrelation_table, "linear_durbinWatson_prop_damage", "The table displays the calculated Durbin-Watson score and its corresponding interpretation based on autocorrelation assumptions. The Durbin-Watson score measures the presence of autocorrelation in the model residuals. Note: The values presented are rounded to two decimal places.", "tbl-linear-durbinWatson-prop-damage")

In [ ]:
residuals_values_df[residuals_values_df["Residuals"]>=0].shape[0] / residuals_values_df.shape[0] * 100

In [ ]:
residuals_values_df[residuals_values_df["Residuals"]<=0].shape[0] / residuals_values_df.shape[0] * 100

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,5))

plt.grid(which='major', color='black', linewidth=0.5, alpha=0.4)
plt.grid(which='minor', color='black', linewidth=0.25, alpha=0.2)

sns.scatterplot(data = residuals_values_df, x = residuals_values_df.index, y="Residuals", alpha=0.05, edgecolor="lightblue", color="None", ax = ax)
plt.axhline(y = 0, color='darkorange', linestyle='--')

plt.ylim(-0.9, 0.9)

plt.xlabel("Data Point Index", fontsize=12)
plt.ylabel("Residuals", fontsize=12)

ax.xaxis.set_minor_locator(MultipleLocator(12.5))
ax.yaxis.set_minor_locator(MultipleLocator(0.1))

plt.title("Distribution of Residuals for Linear Prediction of Proportion of Building Damage")

plt.savefig("../figures/scatter/linear_risiduals_prop_damage.png", dpi=300, facecolor="white", bbox_inches="tight")


plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,5))

plt.grid(which='major', color='black', linewidth=0.5, alpha=0.4)
plt.grid(which='minor', color='black', linewidth=0.25, alpha=0.2)

sns.kdeplot(data = results_summary[results_summary["Predictive R-squared"] > -10], x = "Predictive R-squared", shade=True)

plt.xlabel("Predictive $R^2$ Value", fontsize=12)
plt.ylabel("Density", fontsize=12)


median_value = results_summary[results_summary["Predictive R-squared"] > -10]["Predictive R-squared"].median()
mean_value = results_summary[results_summary["Predictive R-squared"] > -10]["Predictive R-squared"].mean()

plt.axvline(x=median_value, linewidth=2, color='b', label=f"Median: {median_value:.2f}", alpha=0.5, linestyle=":")
plt.axvline(x=mean_value, linewidth=2, color='b', label=f"Mean: {mean_value:.2f}", alpha=0.5, linestyle="--")


ax.xaxis.set_minor_locator(MultipleLocator(0.05))
ax.yaxis.set_minor_locator(MultipleLocator(0.25))

plt.title("Distribution of predictive $R^2$ values for linear model predicting proportion of building burned")

plt.legend(title="Key")
plt.savefig("../figures/distributions/linear_r2_prop_damage.pdf", dpi=300, facecolor="white", bbox_inches="tight")

plt.show()

In [ ]:
rounded_coefficients_df = coefficients_df.describe().T.round(3)
rounded_coefficients_df.columns = rounded_coefficients_df.columns.str.capitalize()
rounded_coefficients_df.drop(columns=['Count'], inplace=True)
rounded_coefficients_df = rounded_coefficients_df.reset_index()
rounded_coefficients_df = rounded_coefficients_df[["index", "Mean", "50%","Std"]]
rounded_coefficients_df = rounded_coefficients_df.rename(columns={"index": "Independent Variable", 
                                                                  "Mean":"Mean Coefficient Value",
                                                                 "50%": "Median Coefficient Value",
                                                                 "Std": "Standard Deviation"})


pandas_to_markdown(rounded_coefficients_df, "linear_coefficients_prop_damage", "Table presenting summary statistics of coefficients obtained from a multiple linear regression analysis. The statistics presented offer insight into the distribution and central tendency of the coefficients, allowing the relationship of independent and target variables to be discovered. Note: The values presented are rounded to three decimal places.", "tbl-linear_coefficients_prop_damage")

In [ ]:
coefficient_df_melt = pd.melt(coefficients_df)

In [ ]:
coefficient_df_melt = coefficient_df_melt[coefficient_df_melt["variable"] != "const"]

In [ ]:
coefficient_df_melt["variable"] = coefficient_df_melt["variable"].replace({"entropy": "arrival_time_entropy"})

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,5))

plt.grid(which='major', color='black', linewidth=0.5, alpha=0.4)
plt.grid(which='minor', color='black', linewidth=0.25, alpha=0.2)

sns.boxplot(x="variable", y="value", data= coefficient_df_melt, ax=ax, color="lightblue")

plt.xlabel("Feature", fontsize=12)
plt.ylabel("Coefficient Value", fontsize=12)


#median_value = results_summary[results_summary["Predictive R-squared"] > -10]["Predictive R-squared"].median()
#mean_value = results_summary[results_summary["Predictive R-squared"] > -10]["Predictive R-squared"].mean()

#plt.axvline(x=median_value, linewidth=2, color='b', label=f"Median: {median_value:.2f}", alpha=0.5, linestyle=":")
#plt.axvline(x=mean_value, linewidth=2, color='b', label=f"Mean: {mean_value:.2f}", alpha=0.5, linestyle="--")

plt.xticks(rotation=45, ha = "right", rotation_mode='anchor')

ax.yaxis.set_minor_locator(MultipleLocator(0.05))

plt.title("Distribution of coefficient values for linear model predicting proportion of building burned")

#plt.legend(title="Key")
plt.savefig("../figures/distributions/linear_constants_prop_damage.pdf", dpi=300, facecolor="white", bbox_inches="tight")

plt.show()

In [ ]:
coefficient_df_melt[coefficient_df_melt["variable"].str.contains("energy")]["value"].mean()

In [ ]:
results_summary["Predictive R-squared"].describe()

In [ ]:
# Assuming you have a DataFrame named 'data' and a column named 'values'
threshold = 0

# Count the number of data points above the threshold
count_above_threshold = results_summary[results_summary["Predictive R-squared"] > threshold].shape[0]

# Calculate the total number of data points
total_data_points = results_summary.shape[0]

# Calculate the percentage
percentage_above_threshold = (count_above_threshold / total_data_points) * 100

print(percentage_above_threshold)

In [ ]:
mean_r2 = results_summary["Predictive R-squared"].mean()
std_r2 = results_summary["Predictive R-squared"].std()

summary_table = pd.DataFrame({
    "Metric": ["Mean R^2^", "Standard Deviation of R^2^"],
    "Value": [mean_r2, std_r2]
})

In [ ]:
summary_table

## Non-linear

In [ ]:
from sklearn.model_selection import cross_val_predict

# Initialize a timer
start_time = time.time()

# Create lists to store results
model_names = []
mse_values = []
rmse_values = []
r2_values = []



# Specify the models and their corresponding constructors
models = [
    #('Neural Network', MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', random_state=42)),
    ('Random Forest', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('Gradient Boosting', GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)),
    ('SVR', SVR(kernel='rbf', C=1.0, epsilon=0.1)),
    ('KNN Regression', KNeighborsRegressor(n_neighbors=5)),
    ('XGBoost', XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3))
]

# Iterate through the models
for model_name, model in models:
    # Create a list to store model results for each iteration
    model_results = []

    for _ in range(num_iterations):
        # Split the data into training and testing sets (80% train, 20% test)
        X_train_nonlinear, X_test_nonlinear, y_train_nonlinear, y_test_nonlinear = train_test_split(X, y, test_size=0.2)

        # Fit the model and make predictions
        model.fit(X_train_nonlinear, y_train_nonlinear)
        y_pred_nonlinear = model.predict(X_test_nonlinear)

        # Calculate metrics
        mse_nonlinear = mean_squared_error(y_test_nonlinear, y_pred_nonlinear)
        rmse_nonlinear = mean_squared_error(y_test_nonlinear, y_pred_nonlinear, squared=False)
        r2_nonlinear = r2_score(y_test_nonlinear, y_pred_nonlinear)

        # Append results to the model_results list
        model_results.append((mse_nonlinear, rmse_nonlinear, r2_nonlinear))

    # Append results to the respective lists
    model_names.extend([model_name] * num_iterations)
    mse_values.extend([result[0] for result in model_results])
    rmse_values.extend([result[1] for result in model_results])
    r2_values.extend([result[2] for result in model_results])

# Create a DataFrame to store the results
nonlinear_model_results_df = pd.DataFrame({
    'Model': model_names,
    'R^2^': r2_values,
    'MSE': mse_values,
    'RMSE': rmse_values
})

# Round the values
nonlinear_model_results_df = nonlinear_model_results_df.round(3)

# Display the DataFrame
nonlinear_model_results_df

# Calculate the duration
end_time = time.time()
duration = end_time - start_time

# Print the duration
print("Total duration: {:.2f} seconds".format(duration))

In [ ]:
nonlinear_model_results_df_1 = nonlinear_model_results_df.copy()

## Comparing Linear and Non-Linear

In [ ]:
linear_model_results_df = results_summary[["Predictive R-squared"]]
linear_model_results_df["Model"] = "Multi-linear \nRegression"
linear_model_results_df = linear_model_results_df.rename(columns={"Predictive R-squared": "R^2^"})
linear_model_results_df = linear_model_results_df[["Model", "R^2^"]]

nonlinear_model_results_df = nonlinear_model_results_df[["Model", "R^2^"]]

all_model_results_df = pd.concat([linear_model_results_df, nonlinear_model_results_df])


# Calculate median R^2^ values for each model
median_r2_group = all_model_results_df.groupby("Model")["R^2^"].median().sort_values()



fig, ax = plt.subplots(1, figsize=(10,5))

plt.grid(which='major', color='black', linewidth=0.5, alpha=0.4)
plt.grid(which='minor', color='black', linewidth=0.25, alpha=0.2)

#plt.axhline(y=0, linewidth=3, color='black', alpha=0.1)

sns.boxplot(data = all_model_results_df, x="Model", y="R^2^", ax=ax, color="lightblue", order = median_r2_group.reset_index()["Model"].to_list())

plt.ylabel("Predictive $R^2$ Value", fontsize=12)
plt.xlabel("Model", fontsize=12)

#plt.legend(title="Key", loc="lower left")

ax.yaxis.set_minor_locator(MultipleLocator(0.125))

plt.title("$R^2$ Value of Models Predicting Proportion of Fire Damage")

plt.ylim(-2, 0.6)

plt.savefig("../figures/distributions/all_model_r2_prop_damage.pdf", dpi=300, facecolor="white", bbox_inches="tight")

plt.show()

In [ ]:
linear_model_results_df = results_summary[["RMSE"]]
linear_model_results_df["Model"] = "Multi-linear \nRegression"
#linear_model_results_df = linear_model_results_df.rename(columns={"Predictive R-squared": "R^2^"})
linear_model_results_df = linear_model_results_df[["Model", "RMSE"]]

nonlinear_model_results_df = nonlinear_model_results_df_1[["Model", "RMSE"]]

all_model_results_df = pd.concat([linear_model_results_df, nonlinear_model_results_df])


# Calculate median R^2^ values for each model
median_r2_group = all_model_results_df.groupby("Model")["RMSE"].median().sort_values()



fig, ax = plt.subplots(1, figsize=(10,5))

plt.grid(which='major', color='black', linewidth=0.5, alpha=0.4)
plt.grid(which='minor', color='black', linewidth=0.25, alpha=0.2)

#plt.axhline(y=0, linewidth=3, color='black', alpha=0.1)

sns.boxplot(data = all_model_results_df, x="Model", y="RMSE", ax=ax, color="lightblue", order = median_r2_group.reset_index()["Model"].to_list())

plt.ylabel("RMSE Value", fontsize=12)
plt.xlabel("Model", fontsize=12)

#plt.legend(title="Key", loc="lower left")

ax.yaxis.set_minor_locator(MultipleLocator(0.005))

plt.title("RMSE Value of Models Predicting Proportion of Fire Damage")

plt.savefig("../figures/distributions/all_model_rmse_prop_damage.pdf", dpi=300, facecolor="white", bbox_inches="tight")

plt.show()

# Linear Model
## Building Damage Area

In [ ]:
# Initialize a timer
start_time = time.time()

# Create lists to store results
building_area_coefficients = []
building_area_r2_scores = []
building_area_r2_score_x = []
building_area_model_r2_scores = []
building_area_p_values = []
building_area_percentage_within_bounds_list = []
real_predicted_values_ba = []
residuals_values_ba = []
building_area_rmse_values = [] 


# Assuming your dataset is in 'inc_df', create the feature matrix (X) and target variable (y)
X_building_area = inc_df[['entropy', 'agency_count', 'after-action', 'casualty info', 'incident info', 'mean_temperature', 'precipitation_sum', 'windspeed', 'energyScore']]
y_building_area = inc_df['fire_damage_area_median_value'][X_building_area.index]

# Handle missing values
X_building_area.dropna(inplace=True)

# Perform one-hot encoding for the categorical column (replace 'energyScore' with the actual column name)
X_building_area = pd.get_dummies(X_building_area, columns=['energyScore'], drop_first=True, dtype=int)

# Add a constant term to the independent variables to account for the intercept in the regression
X_building_area = sm.add_constant(X_building_area)

for _ in range(num_iterations):
    # Split the data into training and testing sets (80% train, 20% test)
    X_train_building_area, X_test_building_area, y_train_building_area, y_test_building_area = train_test_split(X_building_area, y_building_area, test_size=0.2, random_state=np.random.randint(1000))
    
    # Fit the multiple linear regression model
    building_area_model = sm.OLS(y_train_building_area, X_train_building_area).fit()
    
    # Extract coefficients and p-values
    building_area_model_params = building_area_model.params
    building_area_pvals = building_area_model.pvalues
    
    # Get R-squared from the model
    building_area_model_r2 = building_area_model.rsquared
    
    # Generate predictions for the test set
    building_area_test_mean_predictions = building_area_model.predict(X_test_building_area)
    
    # Calculate R-squared based on predictive ability
    building_area_r2_x = 1 - (np.sum((y_test_building_area - building_area_test_mean_predictions) ** 2) / np.sum((y_test_building_area - np.mean(y_test_building_area)) ** 2))
    building_area_r2 = r2_score(y_test_building_area, building_area_test_mean_predictions)
    
    # Calculate the residuals for the test set
    building_area_residuals = y_test_building_area - building_area_test_mean_predictions
    
    # Calculate the standard deviation of residuals (a measure of uncertainty)
    building_area_residual_std = building_area_residuals.std()
    
    # Calculate RMSE for the test set
    building_area_rmse = sqrt(mean_squared_error(y_test_building_area, building_area_test_mean_predictions))
    
    # Append the RMSE value to the list
    building_area_rmse_values.append(building_area_rmse)
    
    # Calculate the range of predictions (within one standard deviation) for the test set
    building_area_test_lower_bound = building_area_test_mean_predictions - building_area_residual_std
    building_area_test_upper_bound = building_area_test_mean_predictions + building_area_residual_std
    
    # Count the number of predictions within the bounds
    building_area_num_within_bounds = ((y_test_building_area >= building_area_test_lower_bound) & (y_test_building_area <= building_area_test_upper_bound)).sum()
    
    # Calculate the percentage of predictions within the bounds
    building_area_percentage_within_bounds = building_area_num_within_bounds / len(y_test_building_area) * 100
    
    # Append results to lists
    building_area_coefficients.append(building_area_model_params)
    building_area_r2_scores.append(building_area_r2)
    building_area_r2_score_x.append(building_area_r2_x)
    building_area_model_r2_scores.append(building_area_model_r2)
    building_area_p_values.append(building_area_pvals)
    building_area_percentage_within_bounds_list.append(building_area_percentage_within_bounds)
    
    
     # Append real and predicted values to the list
    real_predicted_values_ba.append(pd.DataFrame({'Real': y_test_building_area, 'Predicted': building_area_test_mean_predictions}))
    
    # Append the residuals to the list
    residuals_values_ba.append(pd.DataFrame({'Residuals': building_area_residuals}))

# Create a DataFrame to store real and predicted values from each iteration
real_predicted_df_ba = pd.concat(real_predicted_values_ba, axis=0)

residuals_values_df_ba = pd.concat(residuals_values_ba, axis=0)
    

# Create DataFrames to store the results
building_area_coefficients_df = pd.DataFrame(building_area_coefficients)
building_area_p_values_df = pd.DataFrame(building_area_p_values)
building_area_results_summary = pd.DataFrame({
    'Predictive R-squared': building_area_r2_scores,
    'Other Predictive R-squared': building_area_r2_score_x,
    'Model R-squared': building_area_model_r2_scores,
    'RMSE': building_area_rmse_values,  # Add RMSE values here
    'Percentage within Bounds': building_area_percentage_within_bounds_list
})

# Print or analyze the DataFrames as needed
# building_area_coefficients_df
# building_area_p_values_df
# building_area_results_summary

# Calculate the duration
end_time = time.time()
duration = end_time - start_time

# Print the duration
print("Total duration: {:.2f} seconds".format(duration))


In [ ]:
real_predicted_df_ba = real_predicted_df_ba[(real_predicted_df_ba["Predicted"] >= 0) & (real_predicted_df_ba["Predicted"] <= 80)]

In [ ]:
print(str(round(real_predicted_df_ba[real_predicted_df_ba['Predicted'] > real_predicted_df_ba['Real']].shape[0] 
                / real_predicted_df_ba.shape[0] * 100, 2)) +"% overestimated.")

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,5))

# Calculate the slope and intercept for the line of best fit
slope_ba, intercept_ba = np.polyfit(real_predicted_df_ba["Real"], real_predicted_df_ba['Predicted'], 1)

plt.grid(which='major', color='black', linewidth=0.5, alpha=0.4)
plt.grid(which='minor', color='black', linewidth=0.25, alpha=0.2)

ax.xaxis.set_minor_locator(MultipleLocator(6.25))
ax.yaxis.set_minor_locator(MultipleLocator(2.5))

sns.scatterplot(y = real_predicted_df_ba["Predicted"], x = real_predicted_df_ba["Real"], alpha=0.05, edgecolor="lightblue", color="None", ax=ax)
plt.plot([0, 200], [intercept_ba, slope_ba + intercept_ba], color='blue', linestyle='-', label= f'Predicted = {slope_ba:.2f} * Real + {intercept_ba:.2f}')

plt.xlabel("Real", fontsize=12)
plt.ylabel("Predicted", fontsize=12)

plt.xlim(0,real_predicted_df_ba["Real"].max())
plt.ylim(0,real_predicted_df_ba["Predicted"].max())

plt.legend(title="Key")

plt.title('Scatter Plot with Lines of Best Fit for Multilinear regression', y=1.05)
plt.suptitle("Predicting Area of Property Damage", y=0.92, size=10)

plt.savefig("../figures/scatter/linear_real_predicted_area_damage.png", dpi=300, facecolor="white", bbox_inches="tight")

plt.show()

In [ ]:
# Perform the Anderson-Darling test for normal distribution of residuals
p_value_ba = normal_ad(residuals_values_df_ba["Residuals"])[1]
print("p-value from the test - below 0.05 generally means non-normal:", p_value_ba)

# Reporting the normality of the residuals
if p_value_ba < 0.05:
    print("Residuals are not normally distributed")
else:
    print("Residuals are normally distributed")

# Plotting the residuals distribution
fig, ax = plt.subplots(1, figsize=(10,5))
plt.title("Distribution of Residuals")
sns.kdeplot(residuals_values_df_ba["Residuals"], color="lightblue", fill=True, ax=ax)


plt.grid(which='major', color='black', linewidth=0.5, alpha=0.4)
plt.grid(which='minor', color='black', linewidth=0.25, alpha=0.2)

plt.axvline(x=residuals_values_df_ba["Residuals"].median(), linewidth=2, color='b', label=f"Median: {residuals_values_df_ba['Residuals'].median():.2f}", alpha=0.5, linestyle=":")
plt.axvline(x=residuals_values_df_ba["Residuals"].mean(), linewidth=2, color='b', label=f"Mean: {residuals_values_df_ba['Residuals'].mean():.2f}", alpha=0.5, linestyle="--")


ax.xaxis.set_minor_locator(MultipleLocator(12.5))
ax.yaxis.set_minor_locator(MultipleLocator(0.0025))

plt.title("Distribution of risiduals for linear model predicting area of building burned")

plt.legend(title="Key")

plt.savefig("../figures/distributions/linear_risiduals_area_damage.pdf", dpi=300, facecolor="white", bbox_inches="tight")


plt.show()

p_value_thresh_ba=0.05

print()
if p_value > p_value_thresh_ba:
    print('Assumption satisfied')
else:
    print('Assumption not satisfied')
    print()
    print('Confidence intervals will likely be affected')
    print('Try performing nonlinear transformations on variables')


In [ ]:
durbinWatson_ba = durbin_watson(residuals_values_df_ba["Residuals"])

# Interpretation based on the Durbin-Watson score
if durbinWatson_ba < 1.5:
    interpretation_ba = 'Signs of positive autocorrelation. Assumption not satisfied. Consider adding lag variables.'
elif durbinWatson_ba > 2.5:
    interpretation_ba = 'Signs of negative autocorrelation. Assumption not satisfied. Consider adding lag variables.'
else:
    interpretation_ba = 'Little to no autocorrelation. Assumption satisfied.'

# Create a DataFrame
autocorrelation_table_ba = pd.DataFrame({
    'Durbin Watson Score': [durbinWatson_ba],
    'Interpretation': [interpretation_ba]
})

# Round the VIF values to 2 decimal places
autocorrelation_table_ba['Durbin Watson Score'] = autocorrelation_table_ba['Durbin Watson Score'].round(2)

# Print the DataFrame
autocorrelation_table_ba

pandas_to_markdown(autocorrelation_table_ba, "linear_durbinWatson_area_damage", "The table displays the calculated Durbin-Watson score and its corresponding interpretation based on autocorrelation assumptions. The Durbin-Watson score measures the presence of autocorrelation in the model residuals. Note: The values presented are rounded to two decimal places.", "tbl-linear-durbinWatson-area-damage")

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,5))

plt.grid(which='major', color='black', linewidth=0.5, alpha=0.4)
plt.grid(which='minor', color='black', linewidth=0.25, alpha=0.2)

sns.scatterplot(data = residuals_values_df_ba, x = residuals_values_df_ba.index, y="Residuals", alpha=0.05, edgecolor="lightblue", color="None", ax = ax)
plt.axhline(y = 0, color='darkorange', linestyle='--')

plt.ylim(-175, 175)

plt.xlabel("Data Point Index", fontsize=12)
plt.ylabel("Residuals", fontsize=12)

ax.xaxis.set_minor_locator(MultipleLocator(12.5))
#ax.yaxis.set_minor_locator(MultipleLocator(0.1))

plt.title("Distribution of Residuals for Linear Prediction of Area of Building Damage")

plt.savefig("../figures/scatter/linear_risiduals_area_damage.png", dpi=300, facecolor="white", bbox_inches="tight")


plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,5))

plt.grid(which='major', color='black', linewidth=0.5, alpha=0.2)
plt.grid(which='minor', color='black', linewidth=0.25, alpha=0.2)

sns.kdeplot(data = building_area_results_summary, x = "Predictive R-squared", shade=True)

plt.xlabel("Predictive $R^2$ Value", fontsize=12)
plt.ylabel("Density", fontsize=12)


median_r2 = building_area_results_summary["Predictive R-squared"].median()
mean_r2 = building_area_results_summary["Predictive R-squared"].mean()

plt.axvline(x=median_r2, linewidth=2, color='b', label=f"Median: {median_r2:.3f}", alpha=0.5, linestyle=":")
plt.axvline(x=mean_r2, linewidth=2, color='b', label=f"Mean: {mean_r2:.3f}", alpha=0.5, linestyle="--")


ax.xaxis.set_minor_locator(MultipleLocator(0.0625))
ax.yaxis.set_minor_locator(MultipleLocator(0.25))


plt.legend(title="Key")
plt.savefig("../figures/distributions/linear_r2_damage_area.pdf", dpi=300, facecolor="white", bbox_inches="tight")

plt.show()

In [ ]:
building_area_rounded_coefficients_df = building_area_coefficients_df.describe().T.round(3)
building_area_rounded_coefficients_df.columns = building_area_rounded_coefficients_df.columns.str.capitalize()
building_area_rounded_coefficients_df.drop(columns=['Count'], inplace=True)
building_area_rounded_coefficients_df = building_area_rounded_coefficients_df.reset_index()
building_area_rounded_coefficients_df = building_area_rounded_coefficients_df[["index", "Mean", "50%","Std"]]
building_area_rounded_coefficients_df = building_area_rounded_coefficients_df.rename(columns={"index": "Independent Variable", 
                                                                  "Mean":"Mean Coefficient Value",
                                                                 "50%": "Median Coefficient Value",
                                                                 "Std": "Standard Deviation"})



pandas_to_markdown(building_area_rounded_coefficients_df, "linear_coefficients_damage_area", "Table presenting summary statistics of coefficients obtained from a multiple linear regression analysis. The statistics presented offer insight into the distribution and central tendency of the coefficients, allowing the relationship of independent and target variables to be discovered. Note: The values presented are rounded to three decimal places.", "tbl-linear_coefficients_damage_area")

## Non-linear

In [ ]:
# Initialize a timer
start_time_building_area = time.time()

# Create lists to store results
model_names_building_area = []
mse_values_building_area = []
rmse_values_building_area = []
r2_values_building_area = []



# Specify the models and their corresponding constructors
models = [
    #('Neural Network', MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', random_state=42)),
    ('Random Forest', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('Gradient Boosting', GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)),
    ('SVR', SVR(kernel='rbf', C=1.0, epsilon=0.1)),
    ('KNN Regression', KNeighborsRegressor(n_neighbors=5)),
    ('XGBoost', XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=3))
]

# Iterate through the models
for model_name, model in models:
    # Create a list to store model results for each iteration
    model_results_building_area = []

    for _ in range(num_iterations):
        # Split the data into training and testing sets (80% train, 20% test)
        X_train_nonlinear_building_area, X_test_nonlinear_building_area, y_train_nonlinear_building_area, y_test_nonlinear_building_area = train_test_split(X_building_area, y_building_area, test_size=0.2)

        # Fit the model and make predictions
        model.fit(X_train_nonlinear_building_area, y_train_nonlinear_building_area)
        y_pred_nonlinear_building_area = model.predict(X_test_nonlinear_building_area)

        # Calculate metrics
        mse_nonlinear_building_area = mean_squared_error(y_test_nonlinear_building_area, y_pred_nonlinear_building_area)
        rmse_nonlinear_building_area = mean_squared_error(y_test_nonlinear_building_area, y_pred_nonlinear_building_area, squared=False)
        r2_nonlinear_building_area = r2_score(y_test_nonlinear_building_area, y_pred_nonlinear_building_area)

        # Append results to the model_results list
        model_results_building_area.append((mse_nonlinear_building_area, rmse_nonlinear_building_area, r2_nonlinear_building_area))

    # Append results to the respective lists
    model_names_building_area.extend([model_name] * num_iterations)
    mse_values_building_area.extend([result[0] for result in model_results_building_area])
    rmse_values_building_area.extend([result[1] for result in model_results_building_area])
    r2_values_building_area.extend([result[2] for result in model_results_building_area])

# Create a DataFrame to store the results
nonlinear_model_results_df_building_area = pd.DataFrame({
    'Model': model_names_building_area,
    'R^2^': r2_values_building_area,
    'MSE': mse_values_building_area,
    'RMSE': rmse_values_building_area
})

# Round the values
nonlinear_model_results_df_building_area = nonlinear_model_results_df_building_area.round(3)

# Display the DataFrame
nonlinear_model_results_df_building_area

# Calculate the duration
end_time_building_area = time.time()
duration_building_area = end_time_building_area - start_time_building_area

# Print the duration
print("Total duration: {:.2f} seconds".format(duration_building_area))

In [ ]:
nonlinear_model_results_df_building_area_1 = nonlinear_model_results_df_building_area.copy()

In [ ]:
linear_model_results_df_building_area = building_area_results_summary[["Predictive R-squared"]]
linear_model_results_df_building_area["Model"] = "Multi-linear \nRegression"
linear_model_results_df_building_area = linear_model_results_df_building_area.rename(columns={"Predictive R-squared": "R^2^"})
linear_model_results_df_building_area = linear_model_results_df_building_area[["Model", "R^2^"]]

nonlinear_model_results_df_building_area = nonlinear_model_results_df_building_area[["Model", "R^2^"]]

all_model_results_df_building_area = pd.concat([linear_model_results_df_building_area, nonlinear_model_results_df_building_area])


# Calculate median R^2^ values for each model
median_r2_group_building_area = all_model_results_df_building_area.groupby("Model")["R^2^"].median().sort_values()



fig, ax = plt.subplots(1, figsize=(10,5))

plt.grid(which='major', color='black', linewidth=0.5, alpha=0.4)
plt.grid(which='minor', color='black', linewidth=0.25, alpha=0.2)

#plt.axhline(y=0, linewidth=3, color='black', alpha=0.1)

sns.boxplot(data = all_model_results_df_building_area, x="Model", y="R^2^", ax=ax, color="lightblue", order = median_r2_group_building_area.reset_index()["Model"].to_list())

plt.ylabel("Predictive $R^2$ Value", fontsize=12)
plt.xlabel("Model", fontsize=12)

#plt.legend(title="Key", loc="lower left")

ax.yaxis.set_minor_locator(MultipleLocator(0.125))

plt.title("$R^2$ Value of Models Predicting Area of Fire Damage")

plt.ylim(-2, 0.6)

plt.savefig("../figures/distributions/all_model_r2_area_damage.pdf", dpi=300, facecolor="white", bbox_inches="tight")

plt.show()

In [ ]:
linear_model_results_df_building_area_rmse = building_area_results_summary[["RMSE"]]
linear_model_results_df_building_area_rmse["Model"] = "Multi-linear \nRegression"
#linear_model_results_df_building_area = linear_model_results_df_building_area.rename(columns={"Predictive R-squared": "R^2^"})
linear_model_results_df_building_area_rmse = linear_model_results_df_building_area_rmse[["Model", "RMSE"]]

nonlinear_model_results_df_building_area_rmse = nonlinear_model_results_df_building_area_1[["Model", "RMSE"]]

all_model_results_df_building_area_rmse = pd.concat([linear_model_results_df_building_area_rmse, nonlinear_model_results_df_building_area_rmse])


# Calculate median RMSE values for each model
median_r2_group_building_area_rmse = all_model_results_df_building_area_rmse.groupby("Model")["RMSE"].median().sort_values()



fig, ax = plt.subplots(1, figsize=(10,5))

plt.grid(which='major', color='black', linewidth=0.5, alpha=0.4)
plt.grid(which='minor', color='black', linewidth=0.25, alpha=0.2)

#plt.axhline(y=0, linewidth=3, color='black', alpha=0.1)

sns.boxplot(data = all_model_results_df_building_area_rmse, x="Model", y="RMSE", ax=ax, color="lightblue", order = median_r2_group_building_area.reset_index()["Model"].to_list())

plt.ylabel("RMSE Value", fontsize=12)
plt.xlabel("Model", fontsize=12)

ax.yaxis.set_minor_locator(MultipleLocator(1))

plt.title("RMSE Value of Models Predicting Area of Fire Damage")

plt.savefig("../figures/distributions/all_model_rmse_area_damage.pdf", dpi=300, facecolor="white", bbox_inches="tight")

plt.show()

## Model Comparison

In [ ]:
all_model_results_df["Estimating"] = "Proportion"

In [ ]:
all_model_results_df_building_area["Estimating"] = "Area"

In [ ]:
all_results = pd.concat([all_model_results_df_building_area, all_model_results_df])

In [ ]:
linear = all_results[all_results["Model"] == "Multi-linear \nRegression"]

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,5))


plt.grid(which='major', color='black', linewidth=0.5, alpha=0.2)
plt.grid(which='minor', color='black', linewidth=0.25, alpha=0.2)

ax.xaxis.set_minor_locator(MultipleLocator(0.125))
ax.yaxis.set_minor_locator(MultipleLocator(0.125))


sns.kdeplot(data = linear, x="R^2^", hue="Estimating", ax = ax, fill = False)

plt.axvline(x=1, linewidth=2, color='b', label="Area of Damage", alpha=1, linestyle="-")
plt.axvline(x=1, linewidth=2, color='orange', label="Proportion of Damage", alpha=1, linestyle="-")


plt.xlim(-1.5, 0.75)

median_r2_area = linear[linear["Estimating"] == "Area"]["R^2^"].median()
mean_r2_area = linear[linear["Estimating"] == "Area"]["R^2^"].mean()

plt.axvline(x=median_r2, linewidth=2, color='b', label=f"Median: {median_r2_area:.3f}", alpha=0.5, linestyle=":")
plt.axvline(x=mean_r2, linewidth=2, color='b', label=f"Mean: {mean_r2_area:.3f}", alpha=0.5, linestyle="--")

median_r2_prop = linear[linear["Estimating"] == "Proportion"]["R^2^"].median()
mean_r2_prop = linear[linear["Estimating"] == "Proportion"]["R^2^"].mean()

plt.axvline(x=median_r2, linewidth=2, color='orange', label=f"Median: {median_r2_prop:.3f}", alpha=0.5, linestyle=":")
plt.axvline(x=mean_r2, linewidth=2, color='orange', label=f"Mean: {mean_r2_prop:.3f}", alpha=0.5, linestyle="--")

plt.title("KDE Plot Comparing the Performance of Both Models")

plt.legend(title="Key")
plt.savefig("../figures/distributions/all_r2.pdf", dpi=300, facecolor="white", bbox_inches="tight")

plt.show()

In [ ]:
# Calculate the duration
end_time_all = time.time()
duration_all = end_time_all - start_time_all

# Print the duration
print("Total duration: {:.2f} seconds".format(duration_all))